#Necessary Library Imports

In [18]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

#Preparing the Data

#Mount the Google Drive

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Read the data file

In [20]:
text_file = "/content/drive/My Drive/Colab_Data_Files/sin.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==20):
    break

you will receive a package in the mail	ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි
you will receive a confirmation code after completing the registration	ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි
you will receive a discount on your next purchase	ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි
you will receive a phone call from our customer service team	ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතුමක් හම්බවේවි
you will receive a notification when your order is ready for pickup	ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීමක් හම්බවේවි
you will receive a response to your inquiry within 24 hours	ඔයාට පැය 24ක් ඇතුලත ඔයාගෙ විමසීමට පිළිතුරක්  හම්බවේවි
you will receive a gift for your loyalty	ඔයාට ඔයාගේ පක්ෂපාතිත්වය වෙනුවෙන් තෑග්ගක් හම්බවේවි
you will receive an invitation to the event	ඔයාට උත්සවයට ආරාධනාවක් හම්බවේවි
you will receive a refund for the returned item	ඔයාට රිටන් කරන භාණ්ඩය වෙනුවෙන් ආපසු මුදල් ගෙවීමක් හම්බවේවි
you'll receive an email with the details	ඔයාට විස්තර එක්ක

In [22]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

Once upon a time in a small village there lived a young girl named Lily who had a special gift for growing the most beautiful flowers	එක් කලෙක කුඩා ගමක ලිලී නම් තරුණියක් ජීවත් වූ අතර ඇයට ලස්සනම මල් වගා කිරීම සඳහා විශේෂ තෑග්ගක් තිබුණි
In a distant land a brave knight named Sir Arthur embarked on a perilous journey to rescue a kidnapped princess from an evil sorcerer	දුර රටක ශ්‍රීමත් ආතර් නම් නිර්භීත නයිට්වරයා නපුරු මායාකාරයෙකුගෙන් පැහැරගෙන ගිය කුමරියක් බේරා ගැනීමට භයානක ගමනක් ආරම්භ කළේය
Emily a curious explorer set out on an adventure to uncover the hidden treasures of an ancient lost city deep in the Amazon rainforest	කුතුහලය දනවන ගවේෂකයෙකු වන එමිලි ඇමසන් වනාන්තරයේ ගැඹුරින් ගිලිහී ගිය පැරණි නගරයක සැඟවුණු වස්තු සොයා ගැනීමට ත්‍රාසජනක ගමනක් ආරම්භ කළාය
In the peaceful town of Willowbrook a mischievous cat named Oliver had a talent for getting into amusing and unexpected predicaments	සාමකාමී නගරයක් වන විලෝබෲක්හි ඔලිවර් නම් දඟකාර බළලාට විනෝදජනක සහ අනපේක්ෂිත දුෂ්කරතාවන්ට පත්වීමේ දක්ෂතාවයක් ති

#Split the English and Sinhala translation pairs

In [23]:
text_pairs = []
for line in lines:
    english, sinhala = line.split("\t")
    sinhala = "[start] " + sinhala + " [end]"
    text_pairs.append((english, sinhala))
for i in range(3):
  print(random.choice(text_pairs))

("i'll let you pack", '[start] මම ඔයාට බඩු ලෑස්ති කරගන්න දෙන්නම් [end]')
("i'm not like that", '[start] මම එහෙම නෑ [end]')
('i have a great idea to talk as much as we can', '[start] මට පොපියන අදහසක් තියෙනවා අපිට පුළුවන් ගානකට කතා කරගන්න [end]')


#Randomize the data


In [24]:
import random
random.shuffle(text_pairs)

#Spliting the data into training, validation and Testing

In [25]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))


Total sentences: 80684
Training set size: 56480
Validation set size: 12102
Testing set size: 12102


In [26]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

80684

#Removing Punctuations

In [27]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [28]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

#Vectorizing the English and Sinhala text pairs


In [31]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)

#Preparing datasets for the translation task

In [34]:
batch_size = 64
def format_dataset(eng, sin):
    eng = source_vectorization(eng)
    sin = target_vectorization(sin)
    return ({
        "english": eng,
        "sinhala": sin[:, :-1],
    }, sin[:, 1:])
def make_dataset(pairs):
    eng_texts, sin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    sin_texts = list(sin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
    print(f"targets.shape: {targets.shape}")
inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
print(list(train_ds.as_numpy_iterator())[50])


inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
({'english': array([[  32,   20,  674, ...,    0,    0,    0],
       [ 772,    4,  570, ...,    0,    0,    0],
       [1157, 2087,    0, ...,    0,    0,    0],
       ...,
       [1923, 1923, 1078, ...,    0,    0,    0],
       [   4,  305, 1350, ...,    0,    0,    0],
       [ 286,   49,  251, ...,    0,    0,    0]]), 'sinhala': array([[   2,   12,  131, ...,    0,    0,    0],
       [   2, 1152, 2503, ...,    0,    0,    0],
       [   2, 1392, 2758, ...,    0,    0,    0],
       ...,
       [   2, 2744, 2744, ...,    0,    0,    0],
       [   2, 1823, 2643, ...,    0,    0,    0],
       [   2,   56,   10, ...,    0,    0,    0]])}, array([[  12,  131,  102, ...,    0,    0,    0],
       [1152, 2503,   14, ...,    0,    0,    0],
       [1392, 2758,    3, ...,    0,    0,    0],
       ...,
       [2744, 2744, 5554, ...,    0,    0,    0],
       [1823, 2643,   18, ...,    0,    0, 

#Transformer encoder implemented as a subclassed Layer

In [35]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return confi

#The Transformer decoder

In [36]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


#Positional Encoding

In [37]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

#End-to-end Transformer

In [38]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [39]:
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

#Training the sequence-to-sequence Transformer

In [40]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
883/883 [==============================] - 83s 84ms/step - loss: 4.6845 - accuracy: 0.4024 - val_loss: 3.9567 - val_accuracy: 0.4529
Epoch 2/30
883/883 [==============================] - 61s 69ms/step - loss: 3.9171 - accuracy: 0.4669 - val_loss: 3.5533 - val_accuracy: 0.4942
Epoch 3/30
883/883 [==============================] - 61s 69ms/step - loss: 3.6129 - accuracy: 0.4956 - val_loss: 3.3952 - val_accuracy: 0.5096
Epoch 4/30
883/883 [==============================] - 60s 68ms/step - loss: 3.4242 - accuracy: 0.5167 - val_loss: 3.3323 - val_accuracy: 0.5198
Epoch 5/30
883/883 [==============================] - 60s 68ms/step - loss: 3.2941 - accuracy: 0.5325 - val_loss: 3.3412 - val_accuracy: 0.5187
Epoch 6/30
883/883 [==============================] - 60s 68ms/step - loss: 3.1902 - accuracy: 0.5467 - val_loss: 3.3482 - val_accuracy: 0.5220
Epoch 7/30
883/883 [==============================] - 60s 68ms/step - loss: 3.1090 - accuracy: 0.5590 - val_loss: 3.3075 - val_accuracy:

In [64]:
import numpy as np

# Define the vocabulary and index lookup for sinhala
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
not your lucky day
[start] ඔයාගෙ මෝඩ නෑ දවස [end]
-
the clink   did you make it  
[start] මං ඔයාට ඒක කලා කියලා තේරුනාද [end]
-
stop this destruction
[start] නවත්තන්න මේ [UNK] [end]
-
hmm
[start] හ්ම්ම් [end]
-
give up
[start] අතහරින්න [end]
-
he it's complicated
[start] එයා ඒක අවුල් [end]
-
melanie let's go
[start] [UNK] අපි යමු [end]
-
 can you get up    hmmm
[start] ඔයාට පුළුවන්ද හ්ම්ම් හ්ම්ම් [end]
-
work them to death
[start] එයාලාට [UNK] [end]
-
i'm just a bystander
[start] මම නිකම් [UNK] විතරයි [end]
-
you will understand when you see it
[start] ඔයාට ඒක තේරුම් [UNK] නෑ නේද [end]
-
 he wants a picture you tell the truth 
[start] එයාට ඕනේ ඔයා ඇත්ත කියලා අරගන්න [end]
-
gaburin is talking son
[start] [UNK] කතා කරනවා පුතා [end]
-
look i'm not sure
[start] බලන්න මට විශ්වාස නෑ [end]
-
simon kiely's death was an accident
[start] සයිමන් [UNK] පිළිගන්නවා එකක් කියලා [end]
-
what do you want
[start] මොනාද ඕනි [end]
-
black magic
[start] බ්ලැක් මැජික් [end]
-
more  
[start] තව [end]
-
be li